# Demonstration of the BuTools PH package

Set the precision and initialize butools (load all packages)

In [ ]:
%precision %g
%run "~/github/butools/Python/BuToolsInit.py"

First the global *butools.verbose* flag is set to True to obtain more messages from the functions.

In [ ]:
butools.verbose = True

## PH and ME representations

The BuTools PH package offers tools for both phase-type (PH) and matrix-exponential (ME) distributions. Some functions expect a PH representation for the input (MomentsFromPH, PdfFromPH, CdfFromPH, etc.), and some others expect an ME representation (MomentsFromME, PdfFromME, CdfFromME, etc.).

If the global flag called *butools.checkInput* is set to True, these functions enforce the proper representation of their input parameters (the corresponding tolerance is given by the global *butools.checkPrecision* variable).

The following example calls the checking functions on a distribution.

Note that in the Python version of butools, the vector (like $\alpha$ above) must be given by $1\times N$ matrices.

Calling a function that needs a PH representation will fail:

In [ ]:
MomentsFromPH(alpha,A)

However, the ME counterpart of this function runs properly:

In [ ]:
MomentsFromME(alpha,A)

## Moment matching functions

BuTools has many functions to obtain a PH or ME distribution based on the moments. Some of them are able to return a proper result only if the input moment set falls into a given region, while some others are more flexible and have much fewer constraints.

The following function returns a PH(2) based on 3 moments. The 3 moments must fall into a given constraint, otherwise the procedure fails to return a distribution.

In [ ]:
alpha, A = PH2From3Moments ([1,3,20])
print("alpha=", alpha)
print("A=", A)

It can be checked that the moments of the result are the same as the target moments.

In [ ]:
MomentsFromPH(alpha, A)

The extension of the previous matching procedure: matching five moments with a PH(3) 

In [ ]:
alpha, A = PH3From5Moments ([0.9,2.5,20, 500, 22000])
print("alpha=", alpha)
print("A=", A)

If the *graphviz* tool from AT&T is installed, the resulting PH distribution can be visualized by *ImageFromPH*.

In [ ]:
ImageFromPH(alpha,A)

**APHFrom3Moments** is a flexible procedure. It is able to matching *any* three moments with an appropriately large APH (acyclic PH). Let us first try to match moments [1,1.28,8] with PH2From3Moments. It fails.

In [ ]:
alpha, A = PH2From3Moments ([1,1.28,8])

However, APHFrom3Moments is able to obtain a proper PH distribution, although a larger representation.

In [ ]:
alpha, A = APHFrom3Moments ([1,1.28,8])
print("alpha=", alpha)
print("A=", A)

In [ ]:
MomentsFromME(alpha, A)

Let us plot the feasible region of moments for APHFrom3Moments with increasing number of states. As the number of states grows, the entire region is filled.

In [ ]:
def feasibleBounds (n):
    ly = []
    uy = []
    x = []
    for m2 in np.linspace(APH2ndMomentLowerBound(1,n), 1.8, 500):
        # convert to normalized moments
        ml = NormMomsFromMoms ([1, m2, APH3rdMomentLowerBound(1,m2,n)])
        mu = NormMomsFromMoms ([1, m2, APH3rdMomentUpperBound(1,m2,n)])
        # record bounds
        ly.append(ml[2])
        uy.append(min(mu[2],15))
        x.append(ml[1])    
    return (x, ly, uy)

plt.ylim((1.2,3))
plt.xlim((1.1,1.8))
plt.fill_between (*feasibleBounds(6),color="blue")
plt.fill_between (*feasibleBounds(5),color="yellow")
plt.fill_between (*feasibleBounds(4),color="green")
plt.fill_between (*feasibleBounds(3),color="red")
plt.fill_between (*feasibleBounds(2),color="orange")
plt.xlabel("$n_2(=m_2/m_1^2)$")
plt.ylabel("$n_3(=m_3/m_1/m_2)$");

The next procedure, **MEFromMoments** returns a vector-matrix pair matching any number of moments. Unfortunatly, while the moments are mathcing indeed according to the moment formulas, the density function can be negative, thus *MeFromMoments* can return invalid ME distributions.

In [ ]:
v, H = MEFromMoments ([0.9,2.5,20, 500, 22000])
print("v=", v)
print("H=", H)

In [ ]:
MomentsFromME(v, H)

## Advanced representation transformation methods

Let us find a Markovian representation of a non-Markovian one (for instance $(v, H)$). **PHFromME** looks for a solution having the same size. Warning: it is not always possible to find a non-markovian representation of the same size. It can happen that only a larger Markovian representation exists.

In [ ]:
beta, B = PHFromME (v, H)
print("beta=", beta)
print("B=", B)

*PHFromME* terminates successfully. To show that $(\beta,B)$ and $(v,H)$ define the same distribution, we plot it and conclude that they match.

In [ ]:
x = np.linspace(0, 2, 100)
y1 = PdfFromME (v, H, x)
y2 = PdfFromPH (beta, B, x)
plt.plot(x,y1,x,y2);

A more "scientific" way to prove the equivalence of the two representations is to find an appropriate similarity transformation that transforms the one to the other. The transformation matrix can be obtained by the *SimilarityMatrix* function of the *RepTrans* package.

In [ ]:
T = SimilarityMatrix(B,H)
T

Let us now check if $\beta,B$ and $v,H$ are equivalent:

In [ ]:
la.norm(T.I*B*T - H)

In [ ]:
la.norm(beta*T - v)

Now let us take an other example $v,H$, where no Markovian representation of the same size exists.

In [ ]:
v = ml.matrix([[0.2, 0.3, 0.5]])
H = ml.matrix([[-1, 0, 0],[0, -3, 1],[0, -1, -3]])
beta, B = PHFromME (v, H)
print("beta=", beta)
print("B=", B)

Note that there are negative entries in $\beta$ and in the off-diagonals of $B$.
However, there is still hope to obtain a valid Markovian representation.
It has been proven that all ME distributions have a Markovian monocyclic representation.
Luckily we have a function that obtains it, called **MonocyclicPHFromME**. This function is a very strong tool. It is able to convert *any* ME distribution (ok, only those that do not touch the 0 axis apart from point 0) to a PH distribution.

In [ ]:
beta, B = MonocyclicPHFromME (v, H)
print("beta=", beta)
print("B=", B)

Altough it is larger (9 states instead of 3), but at least Markovian. 

In [ ]:
CheckPHRepresentation(beta,B)

Let us check the cdfs now to see if they match.

In [ ]:
alpha = ml.matrix([[0.2, 0.3, 0.5]])
A = ml.matrix([[-1, 0, 0],[0, -3, 1],[0, -1, -3]])

In [ ]:
CheckPHRepresentation(alpha,A)

In [ ]:
CheckMERepresentation(alpha,A)

We can conclude that $\alpha,A$ is not a PH representation, but it is a proper ME representation. (However, this does not mean that it is a valid ME distribution, the non-negativity of the density function is not checked by the CheckMERepresentation function).

In [ ]:
x = np.linspace(0, 2, 100)
y1 = CdfFromME (v, H, x)
y2 = CdfFromPH (beta, B, x)
plt.plot(x,y1,x,y2);

If the return value of *MonocyclicPHFromME* is not interesting, there is *CheckMEPositiveDensity* available to decide if the conversion to PH is possible or not, i.e. if 
the given vector-matrix pair has a non-negative density or not.

In [ ]:
CheckMEPositiveDensity(v,H)

Let us obtain the minimal representation of this redundant representation according to controllability, observability and both respects.

In [ ]:
gammac, Gc = MinimalRepFromME(beta,B,"cont")
print("controllability order = ", Gc.shape[0])
gammao, Go = MinimalRepFromME(beta,B,"obs",1e-4)
print("observability order = ", Go.shape[0])
gamma, G = MinimalRepFromME(beta,B,"obscont") # this is the default, "obscont" can be omitted
print("minimal order = ", G.shape[0])
print("Minimal representation:")
print("gamma=", gamma)
print("G=", G)

This way we got back a $3\times 3$ representation. It is not Markovian, but it is more compact, which is beneficial in many numerical methods that use this distribution.

An other interesting tool is **MEOrderFromMoments**. Based on a set of moments it determines the minimal order of ME distribution that has those moments. For instance, it is often easy to obtain moments from several queueing models: this method can check if there is an underlying PH or ME distribution or not.

In [ ]:
MEOrderFromMoments(MomentsFromPH(beta,B))

It managed to return the right answer. Note however, that this procedure is quite prone to numerical errors. The Mathematica implementation together with multiple (very high) precision arithmetic does not have this problem.

## Randomness

For convenience, there is a procedure to generate a random PH distribution of a given size. Further parameters are the desired order, the desired mean value, and the number of zero entries of the representation (dense random PHs are not diverse enough).

In [ ]:
alpha,A = RandomPH(4, 1.2, 8)
print("alpha=",alpha)
print("A=",A)

(Zero entries can be located in the initial vector, in the generator, and in the absorbing rates as well, which is not instantly visible from the returned parameters)

The *SamplesFromPH* function returns a vector of random samples from a PH distribution. The last parameter is the number of samples requested. The samples can be used in simulations, for instance.